<a href="https://colab.research.google.com/github/ashikshafi08/Learning_Tensorflow/blob/main/Notebooks/Introduction_to_NLP_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Checking what GPU are we running 
!nvidia-smi

Thu May 27 11:11:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy = 'mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


In [4]:
print(tf.__version__)

2.5.0


### Getting the helper functions 


In [5]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-05-27 11:11:14--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-05-27 11:11:14 (111 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [6]:
# Import the needed helper functions for the notebook 
from helper_functions import unzip_data , create_tensorboard_callback , plot_loss_curves , compare_historys

## Get a text dataset 

The dataset we're going to be using is Kaggle's Introduction to NLP dataset (text sampels of tweets labelled as disaster (or) not disaster). 

See the original source here https://www.kaggle.com/c/nlp-getting-started/data

In [7]:
# Download the data from Kaggle 
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzipping the data 
unzip_data('nlp_getting_started.zip')

--2021-05-27 11:11:15--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.128, 142.250.141.128, 2607:f8b0:4023:c03::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-05-27 11:11:15 (112 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Become one with the data 

We will Visualize , explore our test data in here. 

Text datasets come across in many different formats, aside from CSV files we will probably encounter `.txt` file and `.json` files too. Reading the below articles will help in those times, 
* [How to Read and Write Files in Python](https://realpython.com/read-write-files-python/)
* [Working with JSON Data in Python](https://realpython.com/python-json/)

In [8]:
# Importing pandas to look into our csv file 
import pandas as pd 
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# What's inside our train dataframe? 
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
# Getting the first sample of row 1 
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

So our goal now is to build a model to predict the target. Where our targets are it is a disaster or not a disaster. 

In [10]:
# Shuffle the traning dataframe 
train_df_shuffled = train_df.sample(frac = 1 , 
                                    random_state = 42)

# Looking into our shuffled dataframe 
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [11]:
# What does the test dataframe looks like 
test_df.head() 

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


Same as the train dataframe but no targets. In here we are going to use the text column and predict upcoming tweets whether they are Disaster or Not. 

In [12]:
# How many examples of each class? 
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

We can't say our targets are perfectly balanced but it's pretty much a 60-40 split balance between the targets. 

So if we have imabalanced target class refer this https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

In [13]:
# How many total samples in both sets? 

len(train_df) , len(test_df)

(7613, 3263)

In [14]:
# Let's visualize some random training examples. 
import random 

# 5 random index random numbers
# Create random indexes not higher than the total number of samples
random_index = random.randint(0 , len(train_df) - 5) 

# Will return tuples
for row in train_df_shuffled[['text' , 'target']][random_index:random_index+5].itertuples():
  _, text , target = row
  print(f'Target: {target}' , "(real disaster)" if target > 0 else "(not real disaster")
  print(f'Text:\n{text}\n')
  print('----\n')


Target: 0 (not real disaster
Text:
The '1pack 2pack 3pack' line is on fleek if Stretcher is fire.

----

Target: 1 (real disaster)
Text:
70 Years After Atomic Bombs Japan Still Struggles With War Past: The anniversary of the devastation wrought b... http://t.co/aG65u29SGo

----

Target: 0 (not real disaster
Text:
BHAVANA'S MOM HAS CRUSHED EVERYONE'S SOUL

----

Target: 1 (real disaster)
Text:
The Latest: #Wildfire destroys more homes but crews advance - WQOW TV News 18 http://t.co/Hj26SFDdfI

----

Target: 0 (not real disaster
Text:
A Year Later Ferguson Sees Change but Asks if ItÛªs Real http://t.co/H9vmMDEbDx

----



Though we have train and test datasets it's good to create an validation dataset. 

### Split data into training and validation set 

In [15]:
from sklearn.model_selection import train_test_split

# Let's split training data into train and val set
train_sentences , val_sentences , train_labels , val_labels = train_test_split(train_df_shuffled['text'].to_numpy() , 
                                                                               train_df_shuffled['target'].to_numpy() , 
                                                                               test_size = 0.1,  # Use 10% of the training data for validation set 
                                                                               random_state = 42)

In [16]:
# Checking the shapes of our splits 

train_sentences.shape , train_labels.shape , val_sentences.shape , val_labels.shape

((6851,), (6851,), (762,), (762,))

In [17]:
# Number of samples 
print(f'Number of sampels in train set: {len(train_sentences)}')
print(f'Number of sampels in validation set: {len(val_sentences)}')

Number of sampels in train set: 6851
Number of sampels in validation set: 762


In [18]:
# Check the first 10 samples 
train_sentences[:10] , train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

Great now we got to know about how our data looks. The next step would be converting our features into numbers where our targets already in number we don't have to bother about it. 

Next step would be turn the text into numbers! 

## Converting text into numbers 

Alright now the challenge is to convert our text into numbers we can use two techniques they are, 

- **Tokenization**
- **Embeddings**

Let's look into them one by one. 

#### Tokenization

A straight mapping from word or character or sub-word to a numerical value. There are three main levels of tokenization: 

- Using **word-level tokenization** with the sentence 'I love TensorFlow' might result in 'I' being `0` , 'love' being `1` and TensorFlow being `2`. In this case, every word in a sequence considered as a single **token.**
- **Character - level tokenization,** such as converting the letters A-Z to values `1-26`. In this case, every character in a sequence (or) considered as a single token.
- **Sub-word tokenization** is in between word-level and character-level tokenization. It involves breaking individual words into smaller parts and then converting those smaller parts into numbers.

    For example ' my favourite food is pineapple pizza' might become —> " my, fav, avour, rite, fo, oo, od, is, pin, ine, app, le, piz, za". After doing this, these sub-words would then be mapped to a numerical value. In this case every word could be considered multiple tokens. 

#### Embeddings

A embedding is a representation of natural language which can be learned. Representation comes in the form of a **feature vector**. For example, the word 'dance' could be represented by the 5-dimensional vector `[-0.8457 , 0.4559 , -0.3332, 0.9877, 0.1112]`. 

It's important to note here, the size of the feature vector is tuneable (embedding_size). There are two ways to use embeddings: 

- **Create your own embedding** - Once your text has been turned into numbers (requires for an embedding), you can put them through an embedding layer (such as `[tf.keras.layers.Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)`) and embedding representation will be learned during model training.
- **Reuse a pre-learned embedding** - Many pre-trained embeddings exist online. These are pre-trained embeddings have often learned on large corpuses of a text (such as all of Wikipedia) and thus have good underlying representation of natural language. You can use pre-trained embedding to initialize your model and fine-tune it to your own specific task.

Example of **tokenization** (straight mapping from word to number) and **embedding** (richer representation of relationships between tokens).

> Question: What level of tokenzation should I use? What embedding should should I choose?

It depends on your problem. You could try character-level tokenization/embeddings and word-level tokenization/embeddings and see which perform best. You might even want to try stacking them (e.g. combining the outputs of your embedding layers using [`tf.keras.layers.concatenate`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/concatenate) ).

If you're looking for pre-trained word embeddings, [Word2vec embeddings](http://jalammar.github.io/illustrated-word2vec/), [GloVe embeddings](https://nlp.stanford.edu/projects/glove/) and many of the options available on TensorFlow Hub are great places to start.

> Note: Much like searching for a pre-trained computer vision model, you can search for pre-trained word embeddings to use for your problem. Try searching for something like "use pre-trained word embeddings in TensorFlow".

When dealing with a text problem one of the first things you'll have to do before you can build a model is to convert your text to numbers. 

There are few ways to do this, namely: 
* Tokenization - direct mapping of token (a token could be a word or character) to number. 
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned). 

### Text vectorization (tokenization)

In [19]:
# Remind ourselves how our data looks like 
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [20]:
# Import the tokenization layer 
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

How does the `TextVectorization` layer works? 
- standardize each sample (lower casing + punctuation stripping). 
- split each sample into substrings (usually words). 
- recombine substrings into tokens (usually ngrams which will group words). 
- index tokens (assign a unique int value to each token). 
- transform each sample using the index, either into a vector of ints or dense float vector.  

In [21]:
# Use the default TextVectorization parameters 

text_vectorizer = TextVectorization(max_tokens = None , # how many words in the vocabulary (automatically add <OOV>)
                                   standardize = 'lower_and_strip_punctuation' , # Standardize our text data like in image (convert into lower case and strip punctu)
                                   split = 'whitespace', # Split the sequence by whitespace
                                   ngrams = None , # create group of n-words (None will not group them)
                                   output_mode = 'int', # how to map tokens to numbers
                                   output_sequence_length = None, # More like batches (padding) None will pad each sequence to normal sequence
                                   pad_to_max_tokens = True )

We can pad out tweets to the longer sequence but to keep our data small we will find the average words in a sequence and will pad our whole data to it. 

In [22]:
ex = train_sentences[0].split()
len(ex)

7

In [23]:
# Find the average number of tokens (words) in the training tweets. 

round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [24]:
# Setup text vectorization variable 
max_vocab_length = 10000 # Max number of words to have in our vocabulary
max_length = 15 # max length our sequence will be (e.g how many words from Tweet does a model see)

# Creating a instance
text_vectorizer = TextVectorization(max_tokens= max_vocab_length , 
                                    output_mode = 'int' , 
                                    output_sequence_length = max_length)

Now we have a instance of our `TextVectorization` layer now we will have to map this layer to our text data in order to convert them in numerical format. 

We can do this by using `.adapt()` method. 

In [25]:
# Fit the text vectorizer to the training sentence
text_vectorizer.adapt(train_sentences) # Will go through and apply the text vectorization

In [26]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"

# Applying our text vectorization to our above sentence 
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

We can observe that our word got converted into a number and rest 0 is to make sure to (pad) fill up the `output_sequence _length` we mentioned while creating our text vectorization layer. 

In [27]:
# Choose a random sentence from the training dataset and tokenize it 

random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence}\
      \n\nVectorized version:')
text_vectorizer([random_sentence])

Original text:
 Not only are you a mass murderer but at a movie theatre where niggas dropped bread to see a movie? Cmon man.      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  34,  126,   22,   12,    3,  157,  538,   30,   17,    3,  275,
        1951,  221,  936, 1870]])>

Great, thought our sentence is more than 15+ words long but we want to keep our sequence withing the range 0-15. 

We know that `max_tokens` in our text vectorizer layer, it will help us to keep track of the unique words it come across from our data. 

In [28]:
# Get the unique words in the vocabulary 
words_in_vocab = text_vectorizer.get_vocabulary() # Get all of the unique words in our training data

# Most common words in our vocabulary
top_5_words = words_in_vocab[:5] 

# Least common words in our vocabulary
bottom_5_words = words_in_vocab[-5:]

print(f'Number of words in vocab: {len(words_in_vocab)}')
print(f'5 most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


Number of unique words is **10000** its because that's what we set the max_length to. 

UNK - Unknown vocabulary these are words which isn't in our vocabulary. Like 10000 unique words might not cover all of the words in here. 

So if we increase our `max_length` parameter to 20000 then our text vectorizer can handle more unique values and there will be less [UNK] tokens. 

---

Alright let's try out embedding and we know the best part of embedding is it can be learned. 

### Creating an Embedding using a Embedding Layer 

We have got a way to map our text into numbers. How about we find a way and turn those numbers into embeddings. 

To make our embedding, we're going to use a TensorFlow's embedding layer. https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

At first the embedding is created the number are all going to random and we know our embedding layer learns and improves as it goes. Likewise the weights in our model it will get updated to better suit the order of the representation of our words. 

The parameters we care most about for our embedding layer: 
- `input_dim` = the size of our vocabulary
- `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long. 
- `input_lenght` = length of the sequences being passed to the embedding layer (max_length) so it's going to be 15 long. 

In [29]:
# In practice 
from tensorflow.keras import layers 

embedding = layers.Embedding(input_dim= max_vocab_length , # set input shape
                             output_dim = 128,  # output shape
                             input_length = max_length # How long each input is
                             )

# Looking at our embedding layer 
embedding

In [30]:
# Get a random sentence from our training sentence 
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence}')

# Mapping text into numbers (turn into dense vectors of fixed size)
tokenized_form = text_vectorizer([random_sentence]) 
print(f'\n After turning our text into numbers:\n\n {tokenized_form}')

# Using our embedding layer 
print(f'\nApplying the embedding layer to our tokenized vector\n\n {embedding(tokenized_form)}')

Original text:
 I'm not gonna lie I'm kinda ready to attack my Senior year ??????????

 After turning our text into numbers:

 [[  32   34  319 1506   32 1510  924    5  113   13 1987  183    0    0
     0]]

Applying the embedding layer to our tokenized vector

 [[[ 0.01403    0.02782   -0.01688   ...  0.02974   -0.03014    0.01723  ]
  [-0.03833   -0.01423   -0.0005836 ... -0.02573    0.001691   0.04907  ]
  [ 0.02553    0.0482    -0.00809   ... -0.0421    -0.03117    0.04358  ]
  ...
  [-0.005962   0.04236    0.0334    ...  0.04706    0.04657   -0.04935  ]
  [-0.005962   0.04236    0.0334    ...  0.04706    0.04657   -0.04935  ]
  [-0.005962   0.04236    0.0334    ...  0.04706    0.04657   -0.04935  ]]]


In [31]:
sample_embed = embedding(tokenized_form)
sample_embed

<tf.Tensor: shape=(1, 15, 128), dtype=float16, numpy=
array([[[ 0.01403  ,  0.02782  , -0.01688  , ...,  0.02974  ,
         -0.03014  ,  0.01723  ],
        [-0.03833  , -0.01423  , -0.0005836, ..., -0.02573  ,
          0.001691 ,  0.04907  ],
        [ 0.02553  ,  0.0482   , -0.00809  , ..., -0.0421   ,
         -0.03117  ,  0.04358  ],
        ...,
        [-0.005962 ,  0.04236  ,  0.0334   , ...,  0.04706  ,
          0.04657  , -0.04935  ],
        [-0.005962 ,  0.04236  ,  0.0334   , ...,  0.04706  ,
          0.04657  , -0.04935  ],
        [-0.005962 ,  0.04236  ,  0.0334   , ...,  0.04706  ,
          0.04657  , -0.04935  ]]], dtype=float16)>

What is 128? 
Every single token in our sequence are now in the format of 128 long vectors. 

In [32]:
# Check out a single token's embedding 
sample_embed[0][0] , sample_embed[0][0].shape , random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float16, numpy=
 array([ 0.01403 ,  0.02782 , -0.01688 ,  0.04984 , -0.03555 , -0.0373  ,
         0.03696 ,  0.0342  ,  0.000949, -0.041   , -0.003202, -0.04816 ,
        -0.04462 ,  0.002659, -0.00578 ,  0.01288 ,  0.003399,  0.007313,
        -0.0401  ,  0.03072 ,  0.04868 ,  0.0407  ,  0.04752 ,  0.0284  ,
         0.02606 , -0.00323 , -0.01104 , -0.03946 , -0.008194, -0.03418 ,
        -0.0469  ,  0.04318 ,  0.0446  , -0.0338  ,  0.01385 , -0.03946 ,
         0.03354 , -0.04773 , -0.01683 , -0.010826,  0.00502 , -0.02356 ,
        -0.03806 ,  0.01796 , -0.014755, -0.03622 ,  0.01703 ,  0.02184 ,
         0.04065 , -0.04587 , -0.01585 , -0.04886 , -0.04123 , -0.02274 ,
         0.01802 ,  0.0433  , -0.006783,  0.01938 ,  0.0436  , -0.015396,
        -0.003832, -0.02985 ,  0.0448  , -0.02202 , -0.01746 ,  0.0261  ,
         0.03897 , -0.01875 , -0.02516 , -0.0164  ,  0.03348 , -0.0437  ,
        -0.03516 , -0.01564 ,  0.03888 ,  0.005753, -0.01554 , 

Alright next we wil discuss the various modelling experiments we're going to run. 

## Modelling a text dataset (running a series of experiments) 

Once you've got your inputs and outputs prepared, it's a matter of figuring out which machine learning model to build in between them to bridge the gap.

To get a plenty of practice, we're going to build a series of different models, each has its won experiment. We'll then compare the results of each model and see which one performed better. 

We're going to build, 

- **Model 0**: Naive Bayes (common baseline for text based data- tf-idf)
- **Model 1**: Feed-forward neural network (dense model)
- **Model 2**: LSTM model (RNN)
- **Model 3**: GRU model (RNN)
- **Model 4**: Bidirectional-LSTM model (RNN)
- **Model 5**: 1D Convolutional Neural Network
- **Model 6**: TensorFlow Hub Pre-trained Feature Extractor
- **Model 7**: Same as model 6 with 10% of training data.

How are we going to approach all of these? 
Use the standard steps in modelling with TensorFlow: 
- Create a Model
- Build a Model 
- Fit a model 
- Evaluate our model

Let's build a Non-deeplearning model to be more specific a Naive Bayes model from scikit-learn

Let's experiment before the video 


In [33]:
# Checking the shapes of our splits 
train_sentences.shape , train_labels.shape , val_sentences.shape , val_labels.shape

((6851,), (6851,), (762,), (762,))

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Creating a instance
tfidf = TfidfVectorizer()

# Fitting our data to our TfidfVectorizer 
tf_transformer = tfidf.fit(train_sentences)

# Applying the transformation
train_sen_trans = tf_transformer.transform(train_sentences)

# Checking the shape 
train_sen_trans.shape

(6851, 20076)

In [35]:
train_sen_trans[:1]

<1x20076 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [36]:
# Importing the naive bayes model for our classified 
from sklearn.naive_bayes import MultinomialNB

# Creating a instance of our model 
clf_naive = MultinomialNB()

# Fitting the data
clf_naive.fit(train_sen_trans , train_labels)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### Model 0: Getting a baseline 

To create our baseline, we'll use Sklearn's multinomal Naive Bayes using the TF-IDF formula to convert our words to numbers. 

> 🔑 **Note**: It's common practice to use non-DL algorithms as a baseline because of their speed and then later using the DL to see if we can improve. 



In [37]:
train_labels

array([0, 0, 1, ..., 1, 1, 0])

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer # (turn text into numbers)
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline 

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                  ('tfidf' , TfidfVectorizer()) , # Convert words to numbers using tfidf
                  ('clf' , MultinomialNB()), # Model the text 
                ])

# Fit the pipeline to the training data 
model_0.fit(train_sentences , train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [39]:
# Evaluate our baseline model 
# Default evaluation metrics is accuracy
baseline_score = model_0.score(val_sentences , val_labels)
print(f'Our baseline mdoel achieves an accuracy of: {baseline_score*100:.2f}%')

Our baseline mdoel achieves an accuracy of: 79.27%


In [40]:
# Make predictions 
baseline_preds = model_0.predict(val_sentences)

# First 10 predictions
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

Let's use some of the other evaluation metrics. We will make a handy function which will help us to reduce the hustle of writing out every evaluation metrics. 

In [41]:
def classification_evaluation_metrics(y_true , 
                                      y_preds):
  '''
  Arguments: 
  y_true --> true labels of the data 
  y_preds --> predicted labels of the data 

  Returns: 
  A dictionary of evaluation metrics like precision , recall and f1_score
  '''

  # Let's first import the needed metrics 
  from sklearn.metrics import precision_score , f1_score , accuracy_score , recall_score

  # Creting the metrics 
  accuracy = accuracy_score(y_true , y_preds)
  f1_score = f1_score(y_true , y_preds)
  precision = precision_score(y_true , y_preds)
  recall = recall_score(y_true , y_preds)

  # Now will create a dictionary of these metrics and pack them
  evaluation_dict = {'Accuracy:': accuracy * 100 , 
                     'F1_Score: ': f1_score , 
                     'Precision: ': precision , 
                     'Recall: ': recall }

  # Return our dictionary 
  return evaluation_dict
  

In [128]:
# Using the above function 
naive_baseline_results = classification_evaluation_metrics(val_labels , 
                                                    baseline_preds)

# Looking into the dictionary of evaluation metrics 
naive_baseline_results

{'Accuracy:': 79.26509186351706,
 'F1_Score: ': 0.734006734006734,
 'Precision: ': 0.8861788617886179,
 'Recall: ': 0.6264367816091954}

Great! Our baseline model worked better than we expected. Now let's build a Feed-forward neural network model for our text data.

### Model 1: Feed Forward Neural Network (Dense layers) 


In [43]:
# Create a tensorboard callback (to track the model experiments) 
# New one for each model
from helper_functions import create_tensorboard_callback

# Create a directory to save tensorboard logs 
SAVE_DIR = 'model_logs'

In [44]:
# Build model with the functional API 
from tensorflow.keras import layers 

# Creating our input layer (inputs are 1D strings)
inputs = layers.Input(shape=(1,) , dtype = tf.string)

# Convert strings into numbers and applying word embedding
x = text_vectorizer(inputs) # turn input text into numbers 
x = embedding(x) # Create an embedding of the numberized inputs 

# Ouput layer (want binary outputs so use sigmoid activation function)
outputs = layers.Dense(1, activation='sigmoid')(x) 

# Packing into a model 
model_1 = tf.keras.Model(inputs , outputs , name = 'model_1_dense')

# Getting the model summary 
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
dense (Dense)                (None, 15, 1)             129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


At `embedding_2 (Embedding)` our embedding layers adds a extra dimension 128 so our model's parameters gets increased in numbers.

Because every single tokens (15 tokens) get represented as 128 long feature vector. 

In [45]:
# Compile the model 
model_1.compile(loss = tf.keras.losses.BinaryCrossentropy() , 
                optimizer = tf.keras.optimizers.Adam() , 
                metrics = ['accuracy'])

In [46]:
# Fit the model 
model_1_history = model_1.fit(train_sentences ,
                              train_labels , 
                              epochs = 5 , 
                              validation_data = (val_sentences , val_labels) , 
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR , 
                                                                       experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20210527-111123
Epoch 1/5
215/215 [==============================] - 7s 18ms/step - loss: 0.6491 - accuracy: 0.6396 - val_loss: 0.6322 - val_accuracy: 0.6456
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.5968 - accuracy: 0.6908 - val_loss: 0.6251 - val_accuracy: 0.6502
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.5768 - accuracy: 0.6919 - val_loss: 0.6264 - val_accuracy: 0.6510
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.5681 - accuracy: 0.6926 - val_loss: 0.6306 - val_accuracy: 0.6472
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.5633 - accuracy: 0.6926 - val_loss: 0.6324 - val_accuracy: 0.6487


In [47]:
# Check the results 
model_1.evaluate(val_sentences , val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.6324 - accuracy: 0.6487


[0.632392168045044, 0.6487314105033875]

In [48]:
# Make some predictions and evaluate 
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 15, 1)

In [49]:
model_1_pred_probs[:1]

array([[[0.4895 ],
        [0.4895 ],
        [0.4895 ],
        [0.1896 ],
        [0.5073 ],
        [0.4895 ],
        [0.4895 ],
        [0.4895 ],
        [0.09503],
        [0.4177 ],
        [0.4895 ],
        [0.9395 ],
        [0.04108],
        [0.4895 ],
        [0.4067 ]]], dtype=float16)

We want a single prediction probability for each sample but it looks like it is outputing a prediction probability for each tokens.

To fix this we will be using **GlobalAveragePooling** layer. This will help us to condense the feature vector for each token to one vector.

In [50]:
# Let's build the model again but with GlobalAveragePooling 
from tensorflow.keras import layers 

# Input layer
inputs = layers.Input(shape = (1,) , dtype = tf.string)

# Turn text into numbers and create an word embedding 
x = text_vectorizer(inputs)
x = embedding(x)

# Condense the feature vector for each token to "one vector"
x = layers.GlobalAveragePooling1D()(x)

# Output layer 
outputs = layers.Dense(1 , activation = 'sigmoid')(x)

# Packing into a model 
model_1 = tf.keras.Model(inputs , outputs , name = 'model_1_dense')


In [51]:
# Checking the summary of the model
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [52]:
# Compile the model 
model_1.compile(loss = tf.keras.losses.BinaryCrossentropy() , 
                optimizer = tf.keras.optimizers.Adam() , 
                metrics = 'accuracy')

In [53]:
# Fitting the model 
model_1.fit(train_sentences , 
            train_labels , 
            epochs = 5 , 
            validation_data = (val_sentences , val_labels) , 
            callbacks = create_tensorboard_callback(dir_name= SAVE_DIR , 
                                                    experiment_name = 'model_1_dense_pool_layer'))

Saving TensorBoard log files to: model_logs/model_1_dense_pool_layer/20210527-111143
Epoch 1/5
215/215 [==============================] - 5s 17ms/step - loss: 0.5698 - accuracy: 0.7599 - val_loss: 0.5241 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3997 - accuracy: 0.8526 - val_loss: 0.4630 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3173 - accuracy: 0.8800 - val_loss: 0.4524 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2640 - accuracy: 0.9015 - val_loss: 0.4540 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2246 - accuracy: 0.9196 - val_loss: 0.4701 - val_accuracy: 0.7900


In [54]:
# Checking the results by evaluatiing our model 
model_1.evaluate(val_sentences , val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4701 - accuracy: 0.7900


[0.4701315462589264, 0.7900262475013733]

In [55]:
# Making prediction 
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [56]:
# How our predictions looks now? 
model_1_pred_probs[:10]

array([[0.3225],
       [0.8013],
       [0.997 ],
       [0.163 ],
       [0.1277],
       [0.948 ],
       [0.879 ],
       [0.996 ],
       [0.974 ],
       [0.2834]], dtype=float16)

In [57]:
# Convert model prediction probs to label format 

model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float16, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float16)>

In [58]:
# Calculate our model_1 results 
model_1_results = classification_evaluation_metrics(y_true = val_labels , 
                                                    y_preds = model_1_preds)

model_1_results

{'Accuracy:': 79.00262467191601,
 'F1_Score: ': 0.7492163009404389,
 'Precision: ': 0.8241379310344827,
 'Recall: ': 0.6867816091954023}

In [129]:
# Our baseline metrics 
naive_baseline_results

{'Accuracy:': 79.26509186351706,
 'F1_Score: ': 0.734006734006734,
 'Precision: ': 0.8861788617886179,
 'Recall: ': 0.6264367816091954}

In [130]:
# Comparing our base line model with our deeplearning model
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(naive_baseline_results.values()))

array([False,  True, False,  True])

### Visualizing our model's learned word embeddings with TensorFlow's projector tool

In [61]:
# Get the vocabulary from the text vectorization layer 

# Getting the unique vocabulary our embedding layer learnt
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab) , words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [62]:
# Model 1 summary (inspect the embedding layer)
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


Let's get the weight matrix of our embedding layer 

These are the numerical representation of each token in our training data, which have been learned for 5 epochs (patterns from our embedding layer). 

In [63]:
embed_weights = model_1.get_layer('embedding').get_weights()
embed_weights

[array([[ 0.00703812,  0.07912082,  0.07676528, ...,  0.00101803,
          0.01136775, -0.09358066],
        [ 0.02614142, -0.0195411 ,  0.02847962, ..., -0.05451523,
          0.01106524, -0.0627651 ],
        [-0.0047106 ,  0.01667757,  0.0509136 , ..., -0.01553497,
         -0.04616052, -0.05085025],
        ...,
        [ 0.03201046,  0.02925741, -0.02828186, ...,  0.02464871,
         -0.03872454,  0.04999663],
        [ 0.08245139, -0.11478715,  0.10491349, ..., -0.09466122,
          0.08969917, -0.06350309],
        [ 0.15278974, -0.13968992,  0.10933021, ..., -0.09657987,
          0.16960892, -0.13664432]], dtype=float32)]

Those are the weights learned by our embedding layers. 

In [64]:
# The shape should be same size as vocab and embedding_dim (output_dim of our embedding layer)
embed_weights[0].shape 

(10000, 128)

From above we can understand that,
1000 from every token is embedded into 128 dimension vector. 

Now we've got the embeddig matrix of our model has learned to represent our tokens, let's see how we visualize it. 

To do so, TensorFlow has a handy tool called projector: http://projector.tensorflow.org/_

And TensorFlow also has a guide on Word Embeddings: https://www.tensorflow.org/tutorials/text/word_embeddings

In [65]:
# Create embedding files (we got this from TensorFlow's word embeddings documentation)

import io 
# Creating a vector and metadata file for our embeddings (tokens and words)
out_v = io.open('vectors.tsv' , 'w' , encoding = 'utf-8')
out_m = io.open('metadata.tsv' , 'w' , encoding = 'utf-8')

# Loop through and write our values to the corresponding files we created above 
for index , word in enumerate(words_in_vocab):
  if index == 0:
    continue # Skip 0, its padding
  
  vec = embed_weights[0][index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + '\n')

out_v.close()
out_m.close()




In [66]:
# Download files from Colab to upload to the projector 
try: 
  from google.colab import files 
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception: 
  pass  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

> Resources: To learn more about embedding, 
* Jay Alamars visualized word2vec: https://jalammar.github.io/illustrated-word2vec/
* TensorFlow's Word Embedding guide: https://www.tensorflow.org/tutorials/text/word_embeddings

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of previous inputs to aid the representation of the later input. 

Use information from the past to help you with the future (this is where the term recurrent comes from). In other words, take an input (X) and compute an output (y) based on all previous inputs.

> Resources to look into for learning RNN's 
- [RNN MIT Intro to deeplearning](https://www.youtube.com/watch?v=qjrad0V0uJE&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI&index=2)
- [Understanding LSTMS](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- [Unreasonable effectiveness of RNN](https://karpathy.github.io/2015/05/21/rnn-effectiveness/)



### Model 2: LSTM 

LSTM = long short term memory (one of the most popular LSTM cells). 

Our structure of a RNN typically looks like this: 

```
Input (text) --> Tokenize --> Embedding --> Layers (RNNs/dense) --> Output (label probability)
```


In [67]:
# Create an LSTM model 
from tensorflow.keras import layers 

# Setting up inputs 
inputs = layers.Input(shape = (1,) , dtype = tf.string)

# Converting text into numbers and creating a embedding 
x = text_vectorizer(inputs)
x = embedding(x)
print(f'After embedding: {x.shape}')

# Our LSTM 
#x = layers.LSTM(64 , return_sequences=True)(x) 
# When you're stacking RNN cells together you need to set return_sequences = True
#print(f'Output with return sequence True: {x.shape}') # Output with return sequence True
x = layers.LSTM(64)(x)
#print(f'Output with return sequence False: {x.shape}') # Output with return sequence False
x = layers.Dense(64 , activation = 'relu')(x)

# Initializing our outputs 
outputs = layers.Dense(1 , activation = 'sigmoid')(x)

# Packing into a model 
model_2 = tf.keras.Model(inputs , outputs, name = 'model_2_LSTM')

After embedding: (None, 15, 128)


`inputs`: A 3D tensor with shape `[batch, timesteps, feature]`.

- batch --> None by default
- timestaps --> treat every sequence (word) as a timestamps. 

The default activation function for LSTM is **`tanh`**. 

[LSTM layer tensorflow](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)

In [68]:
# Checking the model summary 
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 1,333,633
Trainable params: 1,333,633
Non-trainable params: 0
____________________________________________

In [69]:
# Compile the model 
model_2.compile(loss = tf.keras.losses.BinaryCrossentropy()  , 
                optimizer = tf.keras.optimizers.Adam() , 
                metrics = ['accuracy'])

In [70]:
# Fit the model 
model_2_history = model_2.fit(train_sentences , 
                              train_labels , 
                              validation_data = (val_sentences , val_labels) , 
                              epochs = 5 , 
                              callbacks = [create_tensorboard_callback(dir_name= SAVE_DIR , 
                                                                       experiment_name = 'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210527-111206
Epoch 1/5
215/215 [==============================] - 10s 22ms/step - loss: 0.1975 - accuracy: 0.9302 - val_loss: 0.5763 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 4s 16ms/step - loss: 0.1333 - accuracy: 0.9508 - val_loss: 0.7486 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.1094 - accuracy: 0.9552 - val_loss: 0.6942 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 4s 16ms/step - loss: 0.0848 - accuracy: 0.9658 - val_loss: 1.1842 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0669 - accuracy: 0.9708 - val_loss: 1.0802 - val_accuracy: 0.7835


In [71]:
# Make predictions with LSTM model 
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[5.665e-03],
       [8.965e-01],
       [1.000e+00],
       [1.484e-01],
       [4.333e-05],
       [1.000e+00],
       [4.775e-01],
       [1.000e+00],
       [1.000e+00],
       [3.560e-01]], dtype=float16)

In [72]:
# Convert model 2 pred probs to labels 
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float16, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 0.], dtype=float16)>

In [73]:
# Calculate model 2 results 
model_2_results = classification_evaluation_metrics(val_labels , 
                                                    model_2_preds)
model_2_results

{'Accuracy:': 78.34645669291339,
 'F1_Score: ': 0.7449768160741885,
 'Precision: ': 0.8060200668896321,
 'Recall: ': 0.6925287356321839}

In [131]:
naive_baseline_results

{'Accuracy:': 79.26509186351706,
 'F1_Score: ': 0.734006734006734,
 'Precision: ': 0.8861788617886179,
 'Recall: ': 0.6264367816091954}

### Model 3: GRU Model 

The GRU cell has similar features to an LSTM cell but has less parameters. 

[Understanding GRU networks](https://towardsdatascience.com/understanding-gru-networks-2ef37df6c9be)

Previously we used a LSTM layers to build our RNN model now we will use a GRU layer also called Gater Recurrent Unit to build our model. 

In [75]:
# Create a GRU model 
inputs = layers.Input(shape = (1,) , dtype = tf.string)

# Converting text into numbers and performing word embeddings. 
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)

# Building our GRU model 
# If you want to stack recurrent layers on top of each other, you need return_sequences = True
x = layers.GRU(64 , activation = 'tanh' , return_sequences=True)(x)
print(x.shape)
x = layers.LSTM(44 , return_sequences= True)(x) 
print(x.shape)
x = layers.GRU(100)(x)
print(x.shape)
x = layers.Dense(64 , activation='relu')(x)

# Global Average Pooling layer
#x = layers.GlobalAveragePooling1D()(x)

# Output layer 
outputs = layers.Dense(1 , activation= 'sigmoid')(x)

# Packing into a model 
test_model_3 = tf.keras.Model(inputs , outputs)

(None, 15, 128)
(None, 15, 64)
(None, 15, 44)
(None, 100)


In [76]:
# Summary of the mdoel 
test_model_3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru (GRU)                    (None, 15, 64)            37248     
_________________________________________________________________
lstm_1 (LSTM)                (None, 15, 44)            19184     
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               43800     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                6464  

Alright we did some tinkering up there, now let's create a model 3 with only one GRU layer

In [77]:
# Creating a GRU model 
from tensorflow.keras import layers 

# Input layer 
inputs = layers.Input(shape = (1,) , dtype = tf.string)

# Converting text into numbers and perform word embeddings 
x = text_vectorizer(inputs)
x = embedding(x)

# Our GRU layer 
#x = layers.GRU(64 , activation= 'tanh' , return_sequences= True)(x)
x = layers.GRU(64)(x)

# Global Average Pooling 
#x = layers.GlobalAveragePooling1D()(x)

# Output layer 
outputs = layers.Dense(1 , activation='sigmoid')(x)

# Packing into a model
model_3 = tf.keras.Model(inputs , outputs) 


In [78]:
# Model summary 
model_3.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                37248     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [79]:
# Compile the model 
model_3.compile(loss = tf.keras.losses.BinaryCrossentropy() , 
                optimizer = tf.keras.optimizers.Adam() , 
                metrics = ['accuracy']) 

In [80]:
# Fit the model 
model_3_history = model_3.fit(train_sentences , 
                              train_labels , 
                              validation_data = (val_sentences , val_labels), 
                              epochs = 5 , 
                              callbacks = [create_tensorboard_callback(dir_name= SAVE_DIR , 
                                                                       experiment_name = 'model_3_GRU')]) 


Saving TensorBoard log files to: model_logs/model_3_GRU/20210527-111232
Epoch 1/5
215/215 [==============================] - 6s 21ms/step - loss: 0.1444 - accuracy: 0.9457 - val_loss: 0.8340 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0799 - accuracy: 0.9721 - val_loss: 0.8272 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0653 - accuracy: 0.9734 - val_loss: 0.9744 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0554 - accuracy: 0.9758 - val_loss: 1.0878 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0512 - accuracy: 0.9774 - val_loss: 1.1599 - val_accuracy: 0.7638


In [81]:
# Making predictions on the val set 
model_3_pred_probs = model_3.predict(val_sentences) 
model_3_pred_probs[:10]

array([[7.734e-04],
       [7.241e-01],
       [1.000e+00],
       [3.181e-01],
       [1.314e-04],
       [9.990e-01],
       [6.265e-01],
       [1.000e+00],
       [1.000e+00],
       [9.072e-01]], dtype=float16)

In [82]:
# Convert model 3 pred probs to labels 
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float16, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float16)>

In [83]:
# Using our evalution metrics dictionary 
model_3_results = classification_evaluation_metrics(val_labels , 
                                                    model_3_preds)

In [84]:
model_3_results

{'Accuracy:': 76.37795275590551,
 'F1_Score: ': 0.724770642201835,
 'Precision: ': 0.7745098039215687,
 'Recall: ': 0.6810344827586207}

### Model 4: Bidirectional RNN Model 

Normal RNN's go from left to right (just like you'd read an English sentence) however, a bidirectional RNN goes from right to left as well as left to right. 

A birdirectional wrapper works for any RNN cells. 

Let's give a try before watching the video.  

In [85]:
# Creating a Bidirectional RNN model 
from tensorflow.keras import layers
# Input layer 
inputs = layers.Input(shape = (1, ) , dtype = tf.string)

# Convert text into numbers and perform word embeddings 
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)

# Adding a Bidirectional RNN layer 
x = layers.Bidirectional(layers.LSTM(64 , return_sequences=True))(x)
print(x.shape)

x = layers.Bidirectional(layers.GRU(64))(x)
print(x.shape)

# Output layer 
outputs = layers.Dense(1 , activation= 'sigmoid')(x)
print(outputs.shape)

# Packing into a model 
test_model_4 = tf.keras.Model(inputs , outputs)

(None, 15, 128)
(None, 15, 128)
(None, 128)
(None, 1)


Here we have added `return_sequence = True` because we want to stack another RNN layer with our current RNN layer. 

In [86]:
# Model summary 
test_model_4.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 15, 128)           98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               74496     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 1,453,441
Trainable params: 1,453,441
Non-trainable params: 0
_________________________________________________

Following the video 


In [87]:
# Creating a Bidirectional RNN model 
from tensorflow.keras import layers
# Input layer 
inputs = layers.Input(shape = (1, ) , dtype = tf.string)

# Convert text into numbers and perform word embeddings 
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)

# Adding a Bidirectional RNN layer 
x = layers.Bidirectional(layers.LSTM(64 , return_sequences=True))(x)
print(x.shape)
x = layers.Bidirectional(layers.GRU(64))(x)
print(x.shape)

# Output layer 
outputs = layers.Dense(1 , activation= 'sigmoid')(x)
print(outputs.shape)

# Packing into a model 
model_4 = tf.keras.Model(inputs , outputs)

(None, 15, 128)
(None, 15, 128)
(None, 128)
(None, 1)


In [88]:
# Model Summary 
model_4.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 15, 128)           98816     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               74496     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 1,453,441
Trainable params: 1,453,441
Non-trainable params: 0
_________________________________________________

Bidirectional wrapper doubles the units so the representation will go from **64 to 128**. 

Bidirectional --> representation gets doubled

In [89]:
# Compile the model 
model_4.compile(loss = tf.keras.losses.BinaryCrossentropy() , 
                optimizer = tf.keras.optimizers.Adam() , 
                metrics = ['accuracy'])

In [90]:
# Fit the model 
model_4_history = model_4.fit(train_sentences , 
                              train_labels , 
                              validation_data = (val_sentences , val_labels) , 
                              epochs = 5 , 
                              callbacks = [create_tensorboard_callback(dir_name= SAVE_DIR , 
                                                                       experiment_name = 'model_4_bidirectional')])


Saving TensorBoard log files to: model_logs/model_4_bidirectional/20210527-111257
Epoch 1/5
215/215 [==============================] - 12s 31ms/step - loss: 0.0956 - accuracy: 0.9650 - val_loss: 0.9826 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0519 - accuracy: 0.9771 - val_loss: 1.1905 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0443 - accuracy: 0.9784 - val_loss: 1.3011 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0419 - accuracy: 0.9801 - val_loss: 1.3300 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0446 - accuracy: 0.9787 - val_loss: 1.2506 - val_accuracy: 0.7638


Usually adding a bidirectional to a sequences increase the trainin time. 

In [91]:
# Make predictions without our model 4 
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[1.9417e-03],
       [8.1592e-01],
       [1.0000e+00],
       [4.8267e-01],
       [2.3544e-05],
       [1.0000e+00],
       [1.4175e-02],
       [1.0000e+00],
       [1.0000e+00],
       [9.9707e-01]], dtype=float16)

In [92]:
# Convert pred probs to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float16, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float16)>

In [93]:
# Calculate the results of our bidirectional model 
model_4_results = classification_evaluation_metrics(val_labels , 
                                                    model_4_preds)
model_4_results

{'Accuracy:': 76.37795275590551,
 'F1_Score: ': 0.7256097560975611,
 'Precision: ': 0.7727272727272727,
 'Recall: ': 0.6839080459770115}

Hmm.. Our bidirectional model is performing worse than our uni-directional models. 

We should emphasize from this that not all experiments will go well. 

## Convolutional Neural Networks for Text (and other types of sequences)

We've used CNN's for images but images are typically 2D (height x width).... however, our text data is 1D. 

Previously we've used Conv2D for our image data but now we're going to use Conv1D. 

The typical structure of a Conv1D model for sequences (in our case, text): 
```
Inputs (text) --> Tokenization --> Embedding --> Layer(s) (typically Conv1D + pooling) --> Outputs (class probabilities)
```

https://stackoverflow.com/questions/42883547/intuitive-understanding-of-1d-2d-and-3d-convolutions-in-convolutional-neural-n

## Model 5: Conv1D



In [94]:
# Test our our embedding layer , COnv1D layer and max pooling (Let's look into inputs and outputs)
from tensorflow.keras import layers
# Turn target sequence into a embedding
embedding_test = embedding(text_vectorizer(['this is a test sentence']))

# Building a Conv1D layer
conv_1d = layers.Conv1D(filters = 32 , 
                        kernel_size = 5 , # this is salso referred to as ngrams of 5 (means group of 5 words at a time)
                        activation = 'relu' , 
                        padding = 'valid')

# Conv1D output 
conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer

# Poolin layer 
max_pool = layers.GlobalMaxPool1D()

# Output of our max pool layer 
# equivalent to get the most important features (or) get the features with highest value
max_pool_output = max_pool(conv_1d_output)

In [95]:
# Checking the shapes 
embedding_test.shape , conv_1d_output.shape , max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [96]:
# Our embedding text
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float16, numpy=
array([[[-0.0248   , -0.02011  , -0.02307  , ..., -0.0331   ,
         -0.01888  , -0.02849  ],
        [-0.03815  , -0.0443   ,  0.04297  , ..., -0.03903  ,
          0.01444  ,  0.0003366],
        [ 0.05692  ,  0.02214  , -0.00827  , ..., -0.03488  ,
          0.04462  , -0.02164  ],
        ...,
        [ 0.02113  ,  0.03625  ,  0.0685   , ..., -0.005318 ,
          0.008736 , -0.0618   ],
        [ 0.02113  ,  0.03625  ,  0.0685   , ..., -0.005318 ,
          0.008736 , -0.0618   ],
        [ 0.02113  ,  0.03625  ,  0.0685   , ..., -0.005318 ,
          0.008736 , -0.0618   ]]], dtype=float16)>

In [97]:
# Our Conv1D output 
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float16, numpy=
array([[[0.0831   , 0.       , 0.058    , 0.01854  , 0.       ,
         0.       , 0.05814  , 0.0508   , 0.       , 0.       ,
         0.       , 0.013695 , 0.0651   , 0.04434  , 0.       ,
         0.       , 0.0059   , 0.05902  , 0.       , 0.       ,
         0.03918  , 0.       , 0.       , 0.1293   , 0.03415  ,
         0.       , 0.       , 0.06604  , 0.04205  , 0.       ,
         0.05276  , 0.00952  ],
        [0.04037  , 0.       , 0.       , 0.0513   , 0.       ,
         0.       , 0.04288  , 0.06976  , 0.01243  , 0.       ,
         0.       , 0.       , 0.       , 0.001576 , 0.       ,
         0.04266  , 0.08203  , 0.       , 0.06165  , 0.12006  ,
         0.       , 0.       , 0.       , 0.       , 0.05103  ,
         0.0167   , 0.00949  , 0.01014  , 0.02933  , 0.01422  ,
         0.06067  , 0.0336   ],
        [0.05133  , 0.       , 0.       , 0.       , 0.       ,
         0.       , 0.       , 0.       , 0.00092  

In [98]:
# Condense our conv1d 
# We take max across [1 , 15 , 64] and condense into 
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float16, numpy=
array([[0.0831   , 0.03363  , 0.0937   , 0.0513   , 0.       , 0.       ,
        0.05814  , 0.06976  , 0.05655  , 0.1033   , 0.01706  , 0.013695 ,
        0.0651   , 0.05685  , 0.06067  , 0.04266  , 0.08203  , 0.05902  ,
        0.117    , 0.12006  , 0.03918  , 0.       , 0.0006714, 0.1293   ,
        0.07684  , 0.0167   , 0.00949  , 0.06604  , 0.05792  , 0.0389   ,
        0.1087   , 0.10834  ]], dtype=float16)>

Taking on the challenge and building a Conv1D model 

In [99]:
# Creating Conv1D sequence model 
inputs = layers.Input(shape = (1, ) , dtype = tf.string)

x = text_vectorizer(inputs)
x = embedding(x)
print(f'Shape after embedding: {x.shape}')

# Our Conv1D layer 
x = layers.Conv1D(filters = 32 , 
                  kernel_size = 5 , 
                  padding = 'same' , 
                  activation = 'relu')(x)
print(f'Shape after Conv1D: {x.shape}')
x = layers.Dense(64 , activation= 'relu')(x)
print(f'Shape after into the Dense layer: {x.shape}')
# GlobalMaxPool1D works better
x = layers.GlobalMaxPool1D()(x)
print(f'Shape after into the GlobalMaxPool layer: {x.shape}')

# Output layer 
outputs = layers.Dense(1 , activation= 'sigmoid')(x)

# Packing into a model 
test_model_5 = tf.keras.Model(inputs , outputs)


Shape after embedding: (None, 15, 128)
Shape after Conv1D: (None, 15, 32)
Shape after into the Dense layer: (None, 15, 64)
Shape after into the GlobalMaxPool layer: (None, 64)


In [100]:
# Compile the model 
test_model_5.compile(loss = tf.keras.losses.BinaryCrossentropy(), 
                   optimizer = tf.keras.optimizers.Adam() ,
                   metrics = ['accuracy'])

In [101]:
# Fitting the model 
test_model_5.fit(train_sentences , 
                 train_labels , 
                 validation_data = (val_sentences , val_labels), 
                 epochs = 5 )

Epoch 1/5
215/215 [==============================] - 6s 17ms/step - loss: 0.1328 - accuracy: 0.9591 - val_loss: 0.9383 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 4s 16ms/step - loss: 0.0654 - accuracy: 0.9745 - val_loss: 0.9850 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0532 - accuracy: 0.9781 - val_loss: 1.1027 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0461 - accuracy: 0.9788 - val_loss: 1.2259 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0418 - accuracy: 0.9831 - val_loss: 1.2172 - val_accuracy: 0.7559


Now following the video 


In [102]:
# Creating Conv1D sequence model 
inputs = layers.Input(shape = (1, ) , dtype = tf.string)

x = text_vectorizer(inputs)
x = embedding(x)
print(f'Shape after embedding: {x.shape}')

# Our Conv1D layer 
x = layers.Conv1D(filters = 32 , 
                  kernel_size = 5 , 
                  padding = 'same' , 
                  activation = 'relu')(x)
print(f'Shape after Conv1D: {x.shape}')
#x = layers.Dense(64 , activation= 'relu')(x)
#print(f'Shape after into the Dense layer: {x.shape}')
# GlobalMaxPool1D works better
x = layers.GlobalMaxPool1D()(x)
print(f'Shape after into the GlobalMaxPool layer: {x.shape}')

# Output layer 
outputs = layers.Dense(1 , activation= 'sigmoid')(x)

# Packing into a model 
model_5 = tf.keras.Model(inputs , outputs , name = 'model_5_conv_1d')


Shape after embedding: (None, 15, 128)
Shape after Conv1D: (None, 15, 32)
Shape after into the GlobalMaxPool layer: (None, 32)


In [103]:
# Compile the model 
model_5.compile(loss = tf.keras.losses.BinaryCrossentropy(), 
                   optimizer = tf.keras.optimizers.Adam() ,
                   metrics = ['accuracy'])

In [104]:
model_5.summary()

Model: "model_5_conv_1d"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 15, 32)            20512     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 1,300,545
Trainable params: 1,300,545
Non-trainable params: 0
_________________________________________

In [105]:
# Fitting the model 
model_5.fit(train_sentences , 
                 train_labels , 
                 validation_data = (val_sentences , val_labels), 
                 epochs = 5 , 
            callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR , 
                                                     experiment_name = 'model_5_1d_conv_layer')])

Saving TensorBoard log files to: model_logs/model_5_1d_conv_layer/20210527-111429
Epoch 1/5
215/215 [==============================] - 5s 20ms/step - loss: 0.1093 - accuracy: 0.9645 - val_loss: 0.8827 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0540 - accuracy: 0.9801 - val_loss: 1.0414 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0483 - accuracy: 0.9785 - val_loss: 1.0894 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0440 - accuracy: 0.9806 - val_loss: 1.1324 - val_accuracy: 0.7467
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0421 - accuracy: 0.9801 - val_loss: 1.1919 - val_accuracy: 0.7572


In [106]:
# Make some predictions with our Conv1D model 
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

array([[0.01456],
       [0.6514 ],
       [1.     ],
       [0.03482],
       [0.     ],
       [0.995  ],
       [0.7383 ],
       [1.     ],
       [1.     ],
       [0.945  ]], dtype=float16)

In [107]:
# Convert model 5 pred probs to labels 
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float16, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float16)>

In [108]:
# Evaluate model 5 predictions 
model_5_results = classification_evaluation_metrics(val_labels , 
                                                    model_5_preds)

In [109]:
model_5_results

{'Accuracy:': 75.7217847769029,
 'F1_Score: ': 0.717557251908397,
 'Precision: ': 0.7654723127035831,
 'Recall: ': 0.6752873563218391}

In [132]:
# Our base line model 
naive_baseline_results

{'Accuracy:': 79.26509186351706,
 'F1_Score: ': 0.734006734006734,
 'Precision: ': 0.8861788617886179,
 'Recall: ': 0.6264367816091954}

##   Using TensorFlow Hub for pretrained word embeddings (transfer learning for NLP)


For our next model, instead of using our own embedding layer, we're going to replace it with a pretrained embedding layer.

We have built few of our own models, now let's use a pretrained one.

Now we're gonna build a feature extractor called USE that is Universal Sentence Encoder.
- **Encoder** —> Encodes sequence into numerical representation.
- **Decoder** —> Decode sequence into a desired output.

More specifically, we're going to be using the [Universal Sentence Encoder](https://tfhub.dev/google/collections/universal-sentence-encoder/1)from TensorFlow Hub (a great resource containing a plethora of pretrained model resources for a variety of tasks).

In [111]:
sample_sentence

"There's a flood in my street!"

In [112]:
# Importing the tensorflow hub 
import tensorflow_hub as hub 
embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

# Getting a glimpse of the pretrained model 
embed_samples = embed([sample_sentence , 
                       'When you call a universal sentence encoder on a sentence, it turns it into a numbers.'])
print(embed_samples)

tf.Tensor(
[[-0.01157027  0.0248591   0.02878048 ... -0.00186125  0.02315824
  -0.01485021]
 [ 0.02950275 -0.08060792 -0.01373591 ... -0.02789219  0.02430333
  -0.00082009]], shape=(2, 512), dtype=float32)


It turned our sentence into a numerical representation. 

In [113]:
# Shape of the embed_sample 
embed_samples[0].shape

TensorShape([512])

**Creating a Keras layer using USE pre-trained layer from tensorflow hub.**

We can use the link of the pre-trained model and use it in a Keras layer and turn this into. feature extractor layer. 

In [114]:
# Creating a Keras layer 
sentence_encoder_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4' , 
                                        input_shape = [] , # Defining a input shape so our layer will take anything sequence as a input (any variable length)
                                        dtype = tf.string , 
                                        trainable = False , 
                                        name = 'USE_Layer'
                                        )

In [120]:
# Create model using Sequential API 
model_6 = tf.keras.Sequential([
  # We don't need a text vectorizer and embedding layer this will take care of the pre processing behind the scenes
  sentence_encoder_layer, 
  #layers.Dense(64 , activation = 'relu'), 
  layers.Dense(1 , activation ='sigmoid')
]  , name = 'model_6_USE')

# Compiling the model 
model_6.compile(loss = tf.keras.losses.BinaryCrossentropy() , 
                optimizer = tf.keras.optimizers.Adam() , 
                metrics = ['accuracy'])
# Model 6 summary 
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE_Layer (KerasLayer)       (None, 512)               256797824 
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 513       
Total params: 256,798,337
Trainable params: 513
Non-trainable params: 256,797,824
_________________________________________________________________


In [121]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE_Layer (KerasLayer)       (None, 512)               256797824 
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 513       
Total params: 256,798,337
Trainable params: 513
Non-trainable params: 256,797,824
_________________________________________________________________


In [123]:
# Train a classifier on top of USE pretrained embeddings 
model_6_history = model_6.fit(train_sentences , 
                              train_labels , 
                              epochs = 5 , 
                              validation_data = (val_sentences , val_labels), 
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR , 
                                                                       experiment_name = 'tf_hub_sentence_encoder')])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20210527-113038
Epoch 1/5
215/215 [==============================] - 7s 23ms/step - loss: 0.6461 - accuracy: 0.7438 - val_loss: 0.6108 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.5798 - accuracy: 0.7884 - val_loss: 0.5625 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.5380 - accuracy: 0.7939 - val_loss: 0.5312 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.5097 - accuracy: 0.7997 - val_loss: 0.5106 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4896 - accuracy: 0.8002 - val_loss: 0.4958 - val_accuracy: 0.7900


In [125]:
# Does it beating our baseline model? 
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:5]

array([[0.36  ],
       [0.6816],
       [0.8647],
       [0.3489],
       [0.657 ]], dtype=float16)

In [126]:
# Convert prediction probabilities to labels 
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float16, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float16)>

In [127]:
# Calculate model 6 performance metrics 
model_6_results = classification_evaluation_metrics(val_labels , 
                                                    model_6_preds )
model_6_results

{'Accuracy:': 79.00262467191601,
 'F1_Score: ': 0.7611940298507464,
 'Precision: ': 0.7919254658385093,
 'Recall: ': 0.7327586206896551}

In [133]:
# Baseline model
naive_baseline_results

{'Accuracy:': 79.26509186351706,
 'F1_Score: ': 0.734006734006734,
 'Precision: ': 0.8861788617886179,
 'Recall: ': 0.6264367816091954}

Adding one dense layer to our pretrained model. We call this is as `improv_model_6`

In [152]:
# Creating the model 6 again 
tf.random.set_seed(42)

improv_model_6 = tf.keras.Sequential([
  sentence_encoder_layer , 
  layers.Dense(64 , activation= 'relu'), 
  layers.Dense(1 , activation = 'sigmoid')
])

# Compiling the model 
improv_model_6.compile(loss = tf.keras.losses.BinaryCrossentropy() , 
                       optimizer = tf.keras.optimizers.Adam() , 
                       metrics = ['accuracy'])

# Fit the model 
improv_model_6.fit(train_sentences , 
                   train_labels , 
                   validation_data = (val_sentences , val_labels) , 
                   epochs = 5 )

Epoch 1/5
215/215 [==============================] - 4s 13ms/step - loss: 0.5050 - accuracy: 0.7808 - val_loss: 0.4474 - val_accuracy: 0.7966
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4143 - accuracy: 0.8149 - val_loss: 0.4355 - val_accuracy: 0.8123
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3992 - accuracy: 0.8221 - val_loss: 0.4310 - val_accuracy: 0.8110
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3914 - accuracy: 0.8257 - val_loss: 0.4264 - val_accuracy: 0.8136
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3846 - accuracy: 0.8302 - val_loss: 0.4282 - val_accuracy: 0.8176


In [153]:
# Model summary 
improv_model_6.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE_Layer (KerasLayer)       (None, 512)               256797824 
_________________________________________________________________
dense_22 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


OHH BOY! Atlast adding one additional dense layer managed to beat our base line model. 

In [154]:
# Getting the prediction probs 
improv_model_6_pred_probs = improv_model_6.predict(val_sentences)
improv_model_6_pred_probs[:5]

array([[0.1556],
       [0.7427],
       [0.991 ],
       [0.2173],
       [0.72  ]], dtype=float16)

In [155]:
# Converting into labels 
improv_model_6_preds = tf.squeeze(tf.round(improv_model_6_pred_probs))
improv_model_6_pred_probs[:10]

array([[0.1556],
       [0.7427],
       [0.991 ],
       [0.2173],
       [0.72  ],
       [0.6772],
       [0.9863],
       [0.978 ],
       [0.93  ],
       [0.0883]], dtype=float16)

In [156]:
# Getting our classification report 
improv_model_6_results = classification_evaluation_metrics(val_labels , 
                                                           improv_model_6_preds)
improv_model_6_results

{'Accuracy:': 81.75853018372703,
 'F1_Score: ': 0.785824345146379,
 'Precision: ': 0.8471760797342193,
 'Recall: ': 0.7327586206896551}

In [145]:
# Our base line results 
naive_baseline_results

{'Accuracy:': 79.26509186351706,
 'F1_Score: ': 0.734006734006734,
 'Precision: ': 0.8861788617886179,
 'Recall: ': 0.6264367816091954}

## Model 7: TF Hub Pretrained USE but with 10% of the training data 

Transfer learning really helps when you don't have a large dataset. To see how our model performs on a smaller dataset, let's replicate `model_6` except train with 10% of the training data. 

>Note: Always experiment with a small portion of your data so that we can run more experiments on our data. 

In [160]:
# Create subsets of 10% of the training data 
train_10_percent = train_df_shuffled[['text' , 'target']].sample(frac = 0.1 , random_state = 42)
print(f'Only the 10% of the train data shuffled: {train_10_percent.shape[0]}') 
print(f'The whole data: {train_df_shuffled.shape[0]}')

Only the 10% of the train data shuffled: 761
The whole data: 7613


In [164]:
# Now the same with train_sentences (scraping just the text)
train_sentences_10_percent = train_10_percent['text'].to_list()
train_sentences_10_percent[:5] , len(train_sentences_10_percent) # 761 that is 10% of the text data

(['DFR EP016 Monthly Meltdown - On Dnbheaven 2015.08.06 http://t.co/EjKRf8N8A8 #Drum and Bass #heavy #nasty http://t.co/SPHWE6wFI5',
  'FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/qZQc8WWwcN via @usatoday',
  'Gunmen kill four in El Salvador bus attack: Suspected Salvadoran gang members killed four people and wounded s... http://t.co/CNtwB6ScZj',
  '@camilacabello97 Internally and externally screaming',
  'Radiation emergency #preparedness starts with knowing to: get inside stay inside and stay tuned http://t.co/RFFPqBAz2F via @CDCgov'],
 761)

In [165]:
# Scraping 10% of the labels 
train_labels_10_percent = train_10_percent['target'].to_list()
len(train_labels_10_percent) 

761

In [168]:
# Check the number of targets in our subset of data 
train_10_percent['target'].value_counts()

0    413
1    348
Name: target, dtype: int64

In [169]:
# Checking if the ratio is similar (or very close)
train_df_shuffled['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

While creating a subset of our data we should pay real attention to the distribution is fairly close or equal distribution. 

**Building a similar model to `model_6` but with only 10% of the data**

In [170]:
# Trying to build the sentence encoder layer with no input shape and dtype 
func_sentence_encoder_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4' , 
                                             trainable = False , 
                                             name = 'Functional_USE_layer')

To fix the shape error https://stackoverflow.com/questions/60634839/using-tf-hub-keraslayer-in-tf-2-1-functional-api-throws-valueerror-python-input

In [174]:
# Creating model_7 (Using functional API)
inputs = layers.Input(shape=() , dtype = tf.string , name = 'input_layer')

# Getting our functional sentence encoder in our model
x = func_sentence_encoder_layer(inputs)
x = layers.Dense(64 , activation=  'relu')(x)

# Our ouputs 
outputs = layers.Dense(1 , activation = 'sigmoid' , name = 'output_layer')(x)

# Packing into a model 
functional_model_7 = tf.keras.Model(inputs , outputs , name = 'functional_model_7')

# Compiling the model 
functional_model_7.compile(loss = tf.keras.losses.BinaryCrossentropy() , 
                           optimizer = tf.keras.optimizers.Adam() , 
                           metrics = ['accuracy'])

# Getting the summary of the model 
functional_model_7.summary()


Model: "functional_model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None,)]                 0         
_________________________________________________________________
Functional_USE_layer (KerasL (None, 512)               256797824 
_________________________________________________________________
dense_26 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [175]:
# Fitting the functional model with only the 10% of the data 
functional_model_7_history = functional_model_7.fit(train_sentences_10_percent , 
                                                    train_labels_10_percent , 
                                                    epochs = 5 , 
                                                    validation_data = (val_sentences , val_labels))

Epoch 1/5
24/24 [==============================] - 3s 34ms/step - loss: 0.6739 - accuracy: 0.6518 - val_loss: 0.6376 - val_accuracy: 0.7782
Epoch 2/5
24/24 [==============================] - 0s 18ms/step - loss: 0.6048 - accuracy: 0.7832 - val_loss: 0.5616 - val_accuracy: 0.7900
Epoch 3/5
24/24 [==============================] - 0s 18ms/step - loss: 0.5292 - accuracy: 0.7963 - val_loss: 0.4914 - val_accuracy: 0.7979
Epoch 4/5
24/24 [==============================] - 0s 18ms/step - loss: 0.4696 - accuracy: 0.8055 - val_loss: 0.4427 - val_accuracy: 0.8189
Epoch 5/5
24/24 [==============================] - 0s 18ms/step - loss: 0.4299 - accuracy: 0.8121 - val_loss: 0.4102 - val_accuracy: 0.8215


Doing the same with a sequential API to see both the results are same

In [176]:
# Building model 7 with a sequential api 

model_7 = tf.keras.Sequential([
  sentence_encoder_layer, 
  layers.Dense(64 , activation ='relu'), 
  layers.Dense(1 , activation = 'sigmoid')
] , name = 'model_7')

# Compiling the model 
model_7.compile(loss = tf.keras.losses.BinaryCrossentropy() , 
                optimizer = tf.keras.optimizers.Adam() , 
                metrics = ['accuracy'])

# Fitting the model 
model_7_history = model_7.fit(train_sentences_10_percent , 
                              train_labels_10_percent  , 
                              validation_data = (val_sentences , val_labels) , 
                              epochs = 5)

Epoch 1/5
24/24 [==============================] - 2s 35ms/step - loss: 0.6687 - accuracy: 0.6623 - val_loss: 0.6285 - val_accuracy: 0.7782
Epoch 2/5
24/24 [==============================] - 0s 19ms/step - loss: 0.5931 - accuracy: 0.7924 - val_loss: 0.5485 - val_accuracy: 0.8058
Epoch 3/5
24/24 [==============================] - 0s 18ms/step - loss: 0.5169 - accuracy: 0.8029 - val_loss: 0.4806 - val_accuracy: 0.8097
Epoch 4/5
24/24 [==============================] - 0s 18ms/step - loss: 0.4610 - accuracy: 0.8121 - val_loss: 0.4359 - val_accuracy: 0.8228
Epoch 5/5
24/24 [==============================] - 0s 18ms/step - loss: 0.4247 - accuracy: 0.8200 - val_loss: 0.4058 - val_accuracy: 0.8281


In [177]:
# Getting the prediction probabilities for both 
functional_model_7_pred_probs = functional_model_7.predict(val_sentences)
model_7_pred_probs = model_7.predict(val_sentences)

In [178]:
# Converting into labels of prediction 
functional_model_7_preds = tf.squeeze(tf.round(functional_model_7_pred_probs))
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))

functional_model_7_preds[:5] , model_7_preds[:5]

(<tf.Tensor: shape=(5,), dtype=float16, numpy=array([0., 1., 1., 0., 1.], dtype=float16)>,
 <tf.Tensor: shape=(5,), dtype=float16, numpy=array([0., 1., 1., 0., 1.], dtype=float16)>)

In [179]:
# Comparing the results with the classification metrics we have 

functional_model_7_results = classification_evaluation_metrics(val_labels , 
                                                               functional_model_7_preds)
model_7_results = classification_evaluation_metrics(val_labels , 
                                                    model_7_preds)

functional_model_7_results

{'Accuracy:': 82.1522309711286,
 'F1_Score: ': 0.8,
 'Precision: ': 0.8192771084337349,
 'Recall: ': 0.7816091954022989}

In [180]:
model_7_results

{'Accuracy:': 82.80839895013123,
 'F1_Score: ': 0.8070692194403535,
 'Precision: ': 0.8277945619335347,
 'Recall: ': 0.7873563218390804}

Alright looking at the video now came to know daniel is using clone model here will give a try!

**Clone Model: Copies only the architecture of the model leaving behind the weights / patterns it learned during the training.** 

In [182]:
# Cloning the model 
clone_model_7 = tf.keras.models.clone_model(model_6)

# Compile the model 
clone_model_7.compile(loss = tf.keras.losses.BinaryCrossentropy() , 
                      optimizer = tf.keras.optimizers.Adam() , 
                      metrics = ['accuracy'] ) 

# Get summary of the model (will be same as model_6)
clone_model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE_Layer (KerasLayer)       (None, 512)               256797824 
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 513       
Total params: 256,798,337
Trainable params: 513
Non-trainable params: 256,797,824
_________________________________________________________________
